# Calculator Sample
This sample will guide throught the whole process of implementing, deploying and calling contract in tezos. The guide assumes, that you walk throught it step by step. Scripts are designed to be executed in sequential order. Recommended approach is to walk through the tutorial and then start experimenting and/or looking up more advanced topics afterwards.



## Tezos Sandbox environment
In this sample we are using tezos sandbox. This instance of tezos is local to you computer only. It's primarily used for internal tezos testing. This platform is usefull for experimentation, however if you would like to test your contracts in environment closer to mainnet, it's recommended to use Alphanet. You can check how to spin up alphanet in the section [working with alphanet](../setup/working_with_alphanet.ipynb).

Currently the tezos sandbox requires scripts, that are part of source distribution of tezos. As a prerequisite it's required to build tezos from sources [source build of tezos and liquidity](../setup/source_install.ipynb). 

For usage of sandbox it is required to start tezos node. In full network tezos is using also other [components](../tezos/technical_architecture.md) like `endorser`, `accuser` and `baker`. These components are not used in the sample bellow. Still for development we would need to bake our blocks. To achive that tezos is providing possibility to bake blocks in sandbox using `tezos-client bake ...` command. Baking needs to be executed after every modification of blockchain. Scripts are designed in a way to execute bake after each modification, however in other tezos networks baking is achived using `baker` component.


### Starting the Node for Sandbox Environment
Following commands will start the node for the sandbox network. Script `tezos-sandboxed-node.sh` has one mandatory parameter - number from 1 to 9. This parameter is used to construct RPC port on which sandbox node will be listening. 

Port generation rule:

| Parameter | RPC Port | Node listening for peers |
| --- | --- | --- |
| 1 | 1873**1** | 1973**1** |
| 2 | 1873**2** | 1973**2** |
| 3 | 1873**3** | 1973**3** |
| 4 | ... |  |


For more info on sandbox mode, please consult [tezos documentation on sandbox](http://tezos.gitlab.io/mainnet/introduction/various.html#use-sandboxed-mode)

For convenience, we will run the sandbox as a daemon. Alternatively it's possible to run it in separate terminal.

One important consideration when running Sandbox is persistence. **Sandbox as initiated by provided scripts will be erased after the node is terminated. This needs to be considered when doing development.**

In [1]:
# Initialisation of variables to be used later in tutorial
export TEZOS_HOME=~/tezos-dev/tezos
export TEZOS_NODE_URL="127.0.0.1:18731"

# This variable will disable disclaimer about used tezos network. For this guide we will 
# disable it to make command outputs less cluttered. However we strongly discourage to 
# use this setting for your regular work. (source : Tezos Source code reverse engineering)
export TEZOS_CLIENT_UNSAFE_DISABLE_DISCLAIMER=yes

In [2]:
cd $TEZOS_HOME

./src/bin_node/tezos-sandboxed-node.sh \
  1 \
  --cors-header="*" \
  --cors-origin="*" \
  --connections 1 \
  1>tezos_sandbox_log.txt 2>&1 &

#Exporting node pid so we can terminate the sandbox node after the tutorial.
export NODE_PID=$! 
echo $NODE_PID > tezos_sandbox_pid.txt

[1] 11512


### Initialize Client for Sandbox
Sandbox environment is running on different TCP ports. To reconfigure client to use these values and also to register accounts available in sandbox, tezos is providing initialization script. It's possible to use `tezos-client` even without initialisation, however in this case [wallets](../tezos/basic_concepts.md#Wallet) linked with testing accounts will be not available.

Script will initialize variables and default configuration for the `tezos-client`. Also the script will register aliases for common commands like `tezos-client`. **The variables are present only in current shell. When opening other shell it's required to rerun the command.** If more then two terminals are required, it's possible to run the script in other terminal again. Please note the number after the shell script. This is node identifier - it should match to number of node in previous section.


In [3]:
#It is required to run following script with the same parameter as the sandbox node was executed (in our case 1)
cd $TEZOS_HOME
eval `./src/bin_client/tezos-init-sandboxed-client.sh 1`

## Tezos address added: tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx
## Tezos address added: tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN
## Tezos address added: tz1faswCTDciRzE4oJ9jn2Vm2dvjeyA9fUzU
## Tezos address added: tz1b7tUupMgCNw2cCLpKTkSD1NZzB5TkP2sv
## Tezos address added: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv
## Tezos address added: tz1TGu6TN5GSez2ndXXeDX6LgUDvLzPLqgYV
## 
## The client is now properly initialized. In the rest of this shell
## session, you might now run `tezos-client` to communicate with a
## tezos node launched with `launch-sandboxed-node 1`. For instance:
## 
##   tezos-client rpc get /chains/main/blocks/head/metadata
## 
## Note: if the current protocol version, as reported by the previous
## command, is "Ps9mPmXaRzmzk35gbAYNCAw6UXdE2qoABTHbN2oEEc1qM7CwT9P", you
## may have to activate in your "sandboxed network" the same economic
## protocol than used by the alphanet by running:
## 
##   tezos-activate-alpha
## 
## Warning: all the client data will be removed when you cl

### Activate of Sandbox blockchain
To start working with the tezos sandbox, it's required to bootstrap blockchain. This can be achieved by calling `tezos-activate-alpha` script.

The script will create genesis block and register testing accounts with some &#xA729; balances in `tezos-client`.

In [4]:
tezos-activate-alpha 

Injected BKy7nVTwTHqB


## Reading Data and Navigation Throught Blockchain
After the creation it's possible to check how the actual blockchain look like. Tezos is providing RESTfull RPC interface. It's possible to check in browser directly or using commandline tools. Full documentation of Tezos RPC is provided in [RPC documentation](http://tezos.gitlab.io/mainnet/tutorials/rpc.html).

Following sample shows how to get head of the blockchain (latest block).

In [5]:
#Enabling BASH debug to see variable expansion
set -x

#Following commands are equivalent
#
#  tezos-client rpc get chains/main/blocks/head
#  curl -s http://<address of rpc>/chains/main/blocks/head
#

curl -s "http://$TEZOS_NODE_URL/chains/main/blocks/head" | jq

#Disabling BASH debug
set +x

+ curl -s http://127.0.0.1:18731/chains/main/blocks/head
+ jq
{
  "protocol": "Ps6mwMrF2ER2s51cp9yYpjDcuzQjsc2yAz8bQsRgdaRxw4Fk95H",
  "chain_id": "NetXgtSLGNJvNye",
  "hash": "BKy7nVTwTHqBjQAqnZPvZtaPmrRv15VzXGqDsTRDcDpKFxcxaSw",
  "header": {
    "level": 1,
    "proto": 1,
    "predecessor": "BLockGenesisGenesisGenesisGenesisGenesisb83baZgbyZe",
    "timestamp": "2019-01-16T09:40:37Z",
    "validation_pass": 0,
    "operations_hash": "LLoZS2LW3rEi7KYU4ouBQtorua37aWWCtpDmv1n2x3xoKi6sVXLWp",
    "fitness": [
      "00",
      "0000000000000001"
    ],
    "context": "CoVA4UW7oRk5ed9AZwphxL7nZoD6o4WUHxzDP6nyjsRTVrLiKpTy",
    "content": {
      "command": "activate",
      "hash": "PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP",
      "fitness": [
        "00",
        "0000000000000001"
      ],
      "protocol_parameters": "000007aeae07000004626f6f7473747261705f6163636f756e7473005a02000004300058000000023000370000006564706b75426b6e5732386e5737324b4736526f48745957377031325436474b

Despite possibility to reach RPC using several tools, for learning we encourage to use `tezos-client`. `tezos-client` does provide `list` command, that enables easy navigation and also shows documentation for individual endpoints. 

In [6]:
tezos-client rpc list 


Available services:
  
  + chains/<chain_id>/
    - GET /chains/<chain_id>/blocks
        Lists known heads of the blockchain sorted with decreasing fitness.
        Optional arguments allows to returns the list of predecessors for
        known heads or the list of predecessors for a given list of blocks. 
    - /chains/<chain_id>/blocks/<block_id> <dynamic>
    - GET /chains/<chain_id>/chain_id
        The chain unique identifier. 
    - GET /chains/<chain_id>/invalid_blocks
        Lists blocks that have been declared invalid along with the errors
        that led to them being declared invalid. 
    - GET /chains/<chain_id>/invalid_blocks/<block_hash>
        The errors that appears during the block (in)validation. 
    - DELETE /chains/<chain_id>/invalid_blocks/<block_hash>
        Remove an invalid block for the tezos storage 
    - /chains/<chain_id>/mempool <dynamic>
  - GET /errors
      Schema for all the RPC errors from the shell 
  + injection/
    - POST /injection/block


Following commands are showing documentation and call of sample endpoint.

In [7]:
tezos-client rpc list chains/main/blocks/head


Available services:
  
  + chains/main/blocks/head/
    + context/
      - GET /chains/main/blocks/head/context/constants
          All constants 
      - GET /chains/main/blocks/head/context/constants/errors
          Schema for all the RPC errors from this protocol version 
      - GET /chains/main/blocks/head/context/contracts
          All existing contracts (including non-empty default contracts). 
      + contracts/<contract_id>/
        - GET /chains/main/blocks/head/context/contracts/<contract_id>/balance
            Access the balance of a contract. 
        - POST /chains/main/blocks/head/context/contracts/<contract_id>/big_map_get
            Access the value associated with a key in the big map storage of
            the contract. 
        - GET /chains/main/blocks/head/context/contracts/<contract_id>/counter
            Access the counter of a contract, if any. 
        - GET /chains/main/blocks/head/context/contracts/<contract_id>/delegatable
            Tells if the con

      - POST /chains/main/blocks/head/helpers/preapply/operations
          Simulate the validation of an operation. 
      + scripts/
        - POST /chains/main/blocks/head/helpers/scripts/pack_data
            Computes the serialized version of some data expression using the
            same algorithm as script instruction PACK 
        - POST /chains/main/blocks/head/helpers/scripts/run_code
            Run a piece of code in the current context 
        - POST /chains/main/blocks/head/helpers/scripts/run_operation
            Run an operation without signature checks 
        - POST /chains/main/blocks/head/helpers/scripts/trace_code
            Run a piece of code in the current context, keeping a trace 
        - POST /chains/main/blocks/head/helpers/scripts/typecheck_code
            Typecheck a piece of code in the current context 
        - POST /chains/main/blocks/head/helpers/scripts/typecheck_data
            Check that some data expression is well formed and of a given
  

In [8]:
tezos-client rpc get chains/main/blocks/head | jq

{
  "protocol": "Ps6mwMrF2ER2s51cp9yYpjDcuzQjsc2yAz8bQsRgdaRxw4Fk95H",
  "chain_id": "NetXgtSLGNJvNye",
  "hash": "BKy7nVTwTHqBjQAqnZPvZtaPmrRv15VzXGqDsTRDcDpKFxcxaSw",
  "header": {
    "level": 1,
    "proto": 1,
    "predecessor": "BLockGenesisGenesisGenesisGenesisGenesisb83baZgbyZe",
    "timestamp": "2019-01-16T09:40:37Z",
    "validation_pass": 0,
    "operations_hash": "LLoZS2LW3rEi7KYU4ouBQtorua37aWWCtpDmv1n2x3xoKi6sVXLWp",
    "fitness": [
      "00",
      "0000000000000001"
    ],
    "context": "CoVA4UW7oRk5ed9AZwphxL7nZoD6o4WUHxzDP6nyjsRTVrLiKpTy",
    "content": {
      "command": "activate",
      "hash": "PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP",
      "fitness": [
        "00",
        "0000000000000001"
      ],
      "protocol_parameters": "000007aeae07000004626f6f7473747261705f6163636f756e7473005a02000004300058000000023000370000006564706b75426b6e5732386e5737324b4736526f48745957377031325436474b63376e4162775958356d385764397344564339796176000231000e00000034

Here is a few of usefull endpoints, that can be checked during the tutorial or working with the blockchain

| Endpoint | RPC endpoint |
| --- | --- |
| Get contracts in blockchain | chains/main/blocks/head/context/contracts |
| Get concrete contract in blockchain | chains/main/blocks/head/context/contracts/<contractId> eg. tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN |
| Get storage of originated contract in blockchain | chains/main/blocks/head/context/contracts/<contractId>/storage |
| Get script of originated contract in blockchain | chains/main/blocks/head/context/contracts/<contractId>/script |


### Basic wallet functions and tez transfers
`tezos-client` is serving as a basic wallet. It's possible to get balance for the contracts and also manage private/public key pairs.

After initialisation of the sandbox environment, we can check contracts registered in `tezos-client` by the init scripts and check few initial balances.

In [9]:
# List of registered accounts
tezos-client list known addresses 
echo ----
echo "bootstrap1 balance:" $(tezos-client get balance for bootstrap1)
echo "bootstrap2 balance:" $(tezos-client get balance for bootstrap2)
echo "..."

activator: tz1TGu6TN5GSez2ndXXeDX6LgUDvLzPLqgYV (unencrypted sk known)
bootstrap5: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv (unencrypted sk known)
bootstrap4: tz1b7tUupMgCNw2cCLpKTkSD1NZzB5TkP2sv (unencrypted sk known)
bootstrap3: tz1faswCTDciRzE4oJ9jn2Vm2dvjeyA9fUzU (unencrypted sk known)
bootstrap2: tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN (unencrypted sk known)
bootstrap1: tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx (unencrypted sk known)
----
bootstrap1 balance: 4000000 ꜩ
bootstrap2 balance: 4000000 ꜩ
...


In [10]:
# Listing of security keys
tezos-client show address bootstrap1 --show-secret  

Hash: tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx
Public Key: edpkuBknW28nW72KG6RoHtYW7p12T6GKc7nAbwYX5m8Wd9sDVC9yav
Secret Key: unencrypted:edsk3gUfUPyBSfrS9CCgmCiQsTCHGkviBDusMxDJstFtojtc1zcpsh


### Testing account creation
For our exercise we will generate new contract. Then we can verify successful creation by showing address and public/private (secret) key.

In [11]:
tezos-client gen keys test_account && echo "Keys generated for test_account"
echo -------
tezos-client show address test_account --show-secret

Keys generated for test_account
-------
Hash: tz1gMNzhDYsa3Q87ZczubTN7NwnWGU1ZbtYE
Public Key: edpkucNU9444vrhCUnRhtZd5Pp8ASurr69X2oh5ed7L9RnSisXCuHG
Secret Key: unencrypted:edsk3mGLL8fVUS6ZJfprZu4maZtxTkNE532rNtSLFu5Js12EGbWyk2


In [12]:
tezos-client show address test_account --show-secret

Hash: tz1gMNzhDYsa3Q87ZczubTN7NwnWGU1ZbtYE
Public Key: edpkucNU9444vrhCUnRhtZd5Pp8ASurr69X2oh5ed7L9RnSisXCuHG
Secret Key: unencrypted:edsk3mGLL8fVUS6ZJfprZu4maZtxTkNE532rNtSLFu5Js12EGbWyk2


Our account don't have any tez. So we can transfer some &#xA729; from `bootstrap5` account.

In [13]:
echo "Balance for test_account: " $(tezos-client get balance for test_account) # no tez yet

# We will call transfer operation, that would move 1000 tez from bootstrap5 account to our newly created test_account.
# In the call we are adding --burn-cap parameter. This is specifying maximum amount of tez, that can be burned during
# execution of the contract. This is the operation fee, that would be given to baker.

# we will run the command as a daemon. We will check output after the process will finish
tezos-client transfer 1000 from bootstrap5 to test_account --burn-cap 0.257 > output.txt &
PROCESS_PID=$!


Balance for test_account:  0 ꜩ
[2] 11576


<span style="color:red">Attempt to inject transfer operation will be blocked, as in our blockchain we don't have any bakers. In case you would be using alphanet, bakers would bake the block with your transaction in few minutes.</span>

You can check the state of your operation using RPC provided by tezos. Until the block is baked, the transaction is stored in [mempool](../tezos/basic_concepts.md#Mempool).

In [14]:
tezos-client rpc get chains/main/mempool/pending_operations 
# curl http://$TEZOS_URL/chains/main/mempool/pending_operations | jq

{ "applied":
    [ { "hash": "op6eaWP9tBJ4S6jNAb6yqQ9qCWV3aeaRrU6btKWyCoG3hCgcUL2",
        "branch": "BKy7nVTwTHqBjQAqnZPvZtaPmrRv15VzXGqDsTRDcDpKFxcxaSw",
        "contents":
          [ { "kind": "transaction",
              "source": "tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv",
              "fee": "1275", "counter": "1", "gas_limit": "10200",
              "storage_limit": "277", "amount": "1000000000",
              "destination": "tz1gMNzhDYsa3Q87ZczubTN7NwnWGU1ZbtYE" } ],
        "signature":
          "sigqGJs7WcDiLCUWS5nu31UrVH4wmnRK51FkgC4GivZ3Wh39u4jbumRa9tpP7YjeqkowqN9VN9z1wZwYrE64YrK19boYBLiw" } ],
  "refused": [], "branch_refused": [], "branch_delayed": [],
  "unprocessed": [] }


To resume the operation, you can **bake the block** using `tezos-client`. When baking using the client, you need to provide account, that will be used as delegate. To learn more about delegates and baking, consult [Liquid Proof Of Stake](../tezos/liquid_proof_of_stake.md) and [Tezos Architecture](../tezos/technical_architecture.md).

Following command will bake the block and use `bootstrap1` as a delegate. We can then check output of the command.

In [15]:
tezos-client bake for bootstrap1
wait $PROCESS_PID

cat output.txt

Injected block BLv8Z7UZwXd5
[2]+  Done                    tezos-client transfer 1000 from bootstrap5 to test_account --burn-cap 0.257 > output.txt
Node is bootstrapped, ready for injecting operations.
Estimated gas: 10100 units (will add 100 for safety)
Estimated storage: 257 bytes added (will add 20 for safety)
Operation successfully injected in the node.
Operation hash: op6eaWP9tBJ4S6jNAb6yqQ9qCWV3aeaRrU6btKWyCoG3hCgcUL2
Waiting for the operation to be included...
Operation found in block: BLv8Z7UZwXd5iTJoe32892vZCpYtvJ8VheDdwF8ijat4caXTSQe (pass: 3, offset: 0)
This sequence of operations was run:
  Manager signed operations:
    From: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv
    Fee to the baker: ꜩ0.001275
    Expected counter: 1
    Gas limit: 10200
    Storage limit: 277 bytes
    Balance updates:
      tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv ........... -ꜩ0.001275
      fees(tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx,0) ... +ꜩ0.001275
    Transaction:
      Amount: ꜩ1000
      From: tz1ddb9NM

When using `tezos-client` we are using aliases registered within the client, however this aliases can be represented 
also by contract hash. Contract hash is also used in output. In the output above you can see contract hashes for our new test account and also for bootstrap5 (tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv).

Following two calls are refering to the same contract.

In [16]:
tezos-client get balance for bootstrap5
tezos-client get balance for tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv

3998999.741725 ꜩ
3998999.741725 ꜩ


We can doublecheck balances for `test_account` and `bootstrap5`. From `bootstrap5` balance also transaction fees are subtracted.

In [17]:
echo "Balance for test_account: $(tezos-client get balance for test_account)"
echo "Balance for bootstrap5: $(tezos-client get balance for bootstrap5)"

Balance for test_account: 1000 ꜩ
Balance for bootstrap5: 3998999.741725 ꜩ


During the guide we are using aliases registered within `tezos-client` for convenience reasons. It's possible to use also account hash/address as well. Following two commands are refering to the same account. 

## Tezos Account Types
Up to this point we were working with contracts, that just held tezzies. These accounts are called implicit within Tezos. Other type of contracts are originated. More information on account types can be found on [Originated accounts and Contracts](http://tezos.gitlab.io/mainnet/introduction/howtouse.html#originated-accounts-and-contracts). In tezos implicit accounts have prefix `tz...` and originated accounts `KT1...`. 

Originated accounts can have Michelson code attached. In this case, they are called smart contracts.

    TBD...more clarification required

## Development of Smart Contracts
At this point we have all prerequisites for preparing contract. 

Smart contracts for Tezos are implemented and using Michelson language. It's low-level stack-based language. We will refer to Michelson in other sections of this guide. We will implement sample contract using high level language Liquidity. Liquidity is provided with command line utility, that can be used to compile, deploy and manage the contracts developed using Liquidity language. Liquidity compiles into Michelson. All features, that are available in Michelson are possible in Liquidity as well.

Liquidity is a language heavily inspired by the OCaml language. Before starting working with Liquidity we advise 
to check some OCaml, F# or ReasonML tutorials to familiarize with language syntax (eg. https://try.ocamlpro.com).

To learn more about Liquidity, refer to [Liquidity website](http://www.liquidity-lang.org).

### Sample Calculator Contract
For beginning we will implement sample contract, that will be managing state and will enable add and subtract operations. Default operation will be add.

Smart contracts are software programs that binds the parties involved in a transaction to an agreement with out any third party being involved. The smart contract on the blockchain guarantees that nothing can be changed about the agreement without the consent of all the parties.

When you deploy smart contract into the blockchain it cannot be changed. It prescribes how it's internal storage should be ammended. When smart contract is called, all resulting storage changes are verified agains the smart contract code.

Each smart contract in liquidity consists of four parts
- version declaration (of liquidity)
- storage definition
- storage initialisation
- entry functions (at least method main is required. This method is called when contract is executed)

Following code shows structure of our initial version of contract.

```ocaml
[%%version 0.4]
(* We will store current state of calculator *) 
type storage = int

(* This function will initialise storage when originating contract *)
let%init storage (init_storage : int) =
  init_storage

let%entry add (parameter : int) storage =
  ( ([] : operation list), storage + parameter )
  
let%entry subtract (parameter : int) storage =
  ( ([] : operation list), storage - parameter )
```

Storage block is defining data structure, that will be representing internal state of the contract. For our sample we will use simple structure of type ```int```.

Function marked with `let%init` represents initialisation function for the storage. The function should return storage data structure (in this case int). Returned value would be used as a initial storage value.

Function main is default entry point for contract. Entry points are taking parameters, storage and returning tuple with list of operations, that should be applied after the contract is executed and storage value after execution of the contract. In our case we are not returning any operations, just incrementing value of internal storage.



In [18]:
#Sample preparation
mkdir -p ~/tezos-dev/calculator-dapp
cd ~/tezos-dev/calculator-dapp

/bin/cat <<EOM >Calculator.liq
[%%version 0.4]
  
type storage = int

let%init storage (init_storage : int) =
  init_storage
  
let%entry add (parameter : int) storage =
  ( ([] : operation list), storage + parameter )
  
let%entry subtract (parameter : int) storage =
  ( ([] : operation list), storage - parameter )
EOM



#### Liquidity Development Tooling
Currently there are not many tools, that support development of liquidity contracts. As the language is very similar to OCaml, we suggest to use editors with OCaml syntax highlighting.

Other option is to use web [editor provided on liquidity website](http://www.liquidity-lang.org/edit/). This editor provides more advanced features like code formatting, code completion and comes also with lot of samples how to use liquidity.

### Contract Simulation and Deployment
#### Running Simulation of Contract using Liquidity
Tezos node can execute the script in Michelson and return result even without storage to blockchain. This can be used to simulate contract call during testing and development.

Liquidity can handle transpilation to Michelson and correct call of the script on our behalf. 

During the simulation, it's required to prepare storage structure before calling the contract entry point.
This can be achieved using following command.

In [19]:
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --init-storage '1'

Main contract Calculator
Evaluated initial storage: 1
Constant initial storage generated in "calculator.liq.init.tz"


This command will call storage function (function where declaration starts with `let%init`) and pass arguments after `--init-storage` parameter. Result of the call is evaluated storage as it will be stored to blockchain, but still in Liquidity. Resulting storage in Michelson format will be stored in *.init.tz file.

Liquidity storage:

    Evaluated initial storage: *1*

Michelson storage:

In [20]:
cat calculator.liq.init.tz

1

Now we have all required to run contract simulation using liquidity. For continuing it's important to understand tezos node functionality. Tezos node is important for managing of the blockchain, but it also acts as an interpreter for the Michelson contracts. Execution of the contract, can be tested even without storing results into blockchain.

Following command will execute liquidity script using tezos node.

In [21]:
#in --run add 1 1 first parameter is name of entry point, 
# second represents input to the entry point, 
# third parameter represents storage
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --run add 1 1
 
echo ------

# call substract parameter:3 storage:10
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --run subtract 3 10

Main contract Calculator
2
# Internal operations: 0
------
Main contract Calculator
7
# Internal operations: 0



#### Direct Liquidity Deployment
***Currently direct deployment is not working. For time being, please follow [deployment using tezos-client](#Running-of-Contract-Using-Tezos-Client).***

To deploy the contract to tezos, we will use following command. As parameters we are passing source account and it's private key. In attribute deploy we will pass parameters required for storage initialisation.

In [22]:
#TZ_HASH=$(tezos-client show address test_account --show-secret | grep Hash | cut -d" " -f2)
#TZ_SECRET=$(tezos-client show address test_account --show-secret | grep Secret | cut -d":" -f3)

#liquidity \
#  --tezos-node $TEZOS_NODE_URL \
#  --source $TZ_HASH \
#  --private-key $TZ_SECRET \
#  --fee 1tz \
#  --amount 3tz \
#  Calculator.liq \
#  --deploy '0' | tee contract_deploy_output.txt
  
#export CONTRACT_HASH=$(grep "New contract" contract_deploy_output.txt | cut -d" " -f3)
#export OPERATION_HASH=$(grep "New contract" contract_deploy_output.txt | cut -d" " -f7)

Before continuing we need to bake the block with operation.

In [23]:
# tezos-client bake for bootstrap1 2>/dev/null

To review storage of the contract we can use liquidity program with command --get-storage. As a parameter we are passing hash of the function.

In [24]:
#liquidity \
# --tezos-node localhost:18731 \
# Calculator.liq \
# --get-storage $CONTRACT_HASH

Now we can start working with the contract.

In [25]:
#liquidity \
# --tezos-node localhost:18732 \
# --private-key $TZ_SECRET \
# Calculator.liq \
# --call $CONTRACT_HASH add '10'

After executing the command we still don't see results of our operation in storage. To see changes, we need to bake the block first. This is specific of sandbox environment. In Alphanet, baking will be handled by dedicated bakers.

In [26]:
#echo "Storage before bake"
#liquidity \
# --tezos-node localhost:18731 \
# Calculator.liq \
# --get-storage $CONTRACT_HASH
 
#tezos-client bake for bootstrap1 2>/dev/null

#echo "Storage after the block is baked"
#liquidity \
# --tezos-node localhost:18731 \
# Calculator.liq \
# --get-storage $CONTRACT_HASH

### Running of Contract Using Tezos Client
For this section we will use `liquidity` as a compiler only. When calling the command with the liquidity program, it will compile the code into the Michelson program. 


#### Compilation of Contract to Michelson
Liquidity is hiding lot of complexity, however in some scenarios it is required to use Michelson. Following sample is showing how to use liquidity in order to create michelson script.

When liquidity script is compiled, storage initialization and contract entry points are provided separately. 

In [27]:
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq
 
echo ------

cat Calculator.liq.tz

echo ------

cat Calculator.liq.initializer.tz

Main contract Calculator
Storage initializer generated in "./calculator.liq.initializer.tz"
File "./calculator.liq.tz" generated
If tezos is compiled, you may want to typecheck with:
  tezos-client typecheck script ./calculator.liq.tz
------
parameter (or :_entries (int %_Liq_entry_add) (int %_Liq_entry_subtract));
storage int;
code { DUP ;
       DIP { CDR @storage_slash_1 } ;
       CAR @parameter_slash_2 ;
       DUP @parameter ;
       IF_LEFT
         { RENAME @parameter_slash_3 ; DUUUP ; ADD ; NIL operation ; PAIR }
         { RENAME @parameter_slash_5 ; DUUUP ; SUB ; NIL operation ; PAIR } ;
       DIP { DROP ; DROP } };
------
parameter int;
storage int;
code { DUP ;
       DIP { CDR @_storage } ;
       CAR @_parameter ;
       DUP ;
       DUP @init_storage ;
       DIP { DROP } ;
       NIL operation ;
       PAIR ;
       DIP { DROP ; DROP } };


Liquidity is providing compilation also for method invocations. This way it simplifies creation of required michelson statements.

In [28]:
# Evaluation of the storage structure - calling of storage function and resulting storage structure is outputed
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --init-storage '1'

echo ------
echo add 1
# Compilation of function call - calling of add entry point with argument "1"
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --data add '1'
 
echo ------
echo subtract 4
# Compilation of function call - calling of subtract entry point with argument "4"
liquidity --tezos-node $TEZOS_NODE_URL \
 Calculator.liq \
 --data subtract '4'

Main contract Calculator
Evaluated initial storage: 1
Constant initial storage generated in "calculator.liq.init.tz"
------
add 1
Main contract Calculator
(Left 1)
------
subtract 4
Main contract Calculator
(Right 4)


Now it's possible to simulate the contract directly in Michelson.

In [29]:
echo "Calling add 10, initial storage 0 (calculator.liq.init.tz)"
tezos-client run script calculator.liq.tz on storage $(<calculator.liq.init.tz) and input '(Left 10)'

echo "Calling subtract 5, initial storage 3"
tezos-client run script calculator.liq.tz on storage 3 and input '(Right 5)'

Calling add 10, initial storage 0 (calculator.liq.init.tz)
storage
  11
emitted operations
  


Calling subtract 5, initial storage 3
storage
  -2
emitted operations
  




#### Deployment of Contract using Tezos Client
For deployment we need to create originated account. This can be achieved using `tezos-client originate contract...` call. During the origination it's required to provide our private/secret key ([command to retrieve](#Testing-account-creation)).

In [30]:
# We are creating new originated contract and registring it's alias Calculator with our tezos-client.
tezos-client originate contract Calculator \
 for test_account \
 transferring 1 from test_account \
 running calculator.liq.tz \
 --init $(<calculator.liq.init.tz) \
 --burn-cap 1 > contract_output.txt &

PROCESS_PID=$!

[2] 11602


Operation is not reflected in tezos blockchain until new block is baked. 

In [31]:
tezos-client bake for bootstrap1

wait $PROCESS_PID
cat contract_output.txt

Injected block BMK16A4YSv3B
[2]+  Done                    tezos-client originate contract Calculator for test_account transferring 1 from test_account running calculator.liq.tz --init $(<calculator.liq.init.tz) --burn-cap 1 > contract_output.txt
Node is bootstrapped, ready for injecting operations.
Estimated gas: 16707 units (will add 100 for safety)
Estimated storage: 555 bytes added (will add 20 for safety)
Operation successfully injected in the node.
Operation hash: ooSe7etkFcGRvndc3vz7t4cZc9ZHLaQY1yVPKyopjP3DV5TL8dT
Waiting for the operation to be included...
Operation found in block: BMK16A4YSv3BKb2UwmdnmVaUqv47WkAKazgdRNLL9k9WUK2gcFL (pass: 3, offset: 0)
This sequence of operations was run:
  Manager signed operations:
    From: tz1gMNzhDYsa3Q87ZczubTN7NwnWGU1ZbtYE
    Fee to the baker: ꜩ0.001257
    Expected counter: 2
    Gas limit: 10000
    Storage limit: 0 bytes
    Balance updates:
      tz1gMNzhDYsa3Q87ZczubTN7NwnWGU1ZbtYE ........... -ꜩ0.001257
      fees(tz1KqTpEZ7Yob7Qb

`tezos-client` registers alias for newly created contract. The alias is matching hash outputed in the originate call above.


In [32]:
CONTRACT_HASH=$(grep "New contract KT" contract_output.txt | cut -d" " -f3)
echo "Contract Hash $CONTRACT_HASH"
echo ------
echo "Calculator listed among aliases"
tezos-client list known contracts

Contract Hash KT1BhGVKFxULPWJB3R3JEkNyBVCsHU4hffYT
------
Calculator listed among aliases
Calculator: KT1BhGVKFxULPWJB3R3JEkNyBVCsHU4hffYT
test_account: tz1gMNzhDYsa3Q87ZczubTN7NwnWGU1ZbtYE
activator: tz1TGu6TN5GSez2ndXXeDX6LgUDvLzPLqgYV
bootstrap5: tz1ddb9NMYHZi5UzPdzTZMYQQZoMub195zgv
bootstrap4: tz1b7tUupMgCNw2cCLpKTkSD1NZzB5TkP2sv
bootstrap3: tz1faswCTDciRzE4oJ9jn2Vm2dvjeyA9fUzU
bootstrap2: tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN
bootstrap1: tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx


### Call of contract using tezos-client
Calling of smart contract is the same as transfering balance. When calling smart contract we are providing additional arguments, that should be in Michelson - refer to [Michelson compilation](#Compilation-of-Contract-to-Michelson) for sample how to get to Michelson function call for liquidity function.

In [33]:
#Calling our contract, entry point add (using parameter generated by liquidity compiled function call)
tezos-client \
 transfer 0 from test_account to $CONTRACT_HASH --arg "(Left 10)" > output.txt &
PROCESS_PID=$!

#Baking manually
tezos-client bake for bootstrap1  #baking the block
wait $PROCESS_PID
cat output.txt

[2] 11610
Injected block BKqvSTj1HjdQ
[2]+  Done                    tezos-client transfer 0 from test_account to $CONTRACT_HASH --arg "(Left 10)" > output.txt
Node is bootstrapped, ready for injecting operations.
Estimated gas: 15378 units (will add 100 for safety)
Estimated storage: no bytes added
Operation successfully injected in the node.
Operation hash: opJJjuvjLbLGnJ8b4N6wVHMNiSGZm2qau1SqW4895Qc9THKEj5a
Waiting for the operation to be included...
Operation found in block: BKqvSTj1HjdQHP6jcrem3D5YmwX41LQcyFyJkz3R8xV1of4ou7G (pass: 3, offset: 0)
This sequence of operations was run:
  Manager signed operations:
    From: tz1gMNzhDYsa3Q87ZczubTN7NwnWGU1ZbtYE
    Fee to the baker: ꜩ0.001807
    Expected counter: 4
    Gas limit: 15478
    Storage limit: 0 bytes
    Balance updates:
      tz1gMNzhDYsa3Q87ZczubTN7NwnWGU1ZbtYE ........... -ꜩ0.001807
      fees(tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx,0) ... +ꜩ0.001807
    Transaction:
      Amount: ꜩ0
      From: tz1gMNzhDYsa3Q87ZczubTN7NwnW

We can verify contract call by querying storage in blockchain directly.

In [34]:
tezos-client get script storage for Calculator
# curl -s http://$TEZOS_NODE_URL/chains/main/blocks/head/context/contracts/$CONTRACT_HASH/storage | jq

11


Alternatively we can check the block in which the operation was included.

In [35]:
curl -s http://$TEZOS_NODE_URL/chains/main/blocks/head | jq

{
  "protocol": "PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP",
  "chain_id": "NetXgtSLGNJvNye",
  "hash": "BKqvSTj1HjdQHP6jcrem3D5YmwX41LQcyFyJkz3R8xV1of4ou7G",
  "header": {
    "level": 4,
    "proto": 1,
    "predecessor": "BMK16A4YSv3BKb2UwmdnmVaUqv47WkAKazgdRNLL9k9WUK2gcFL",
    "timestamp": "2019-01-16T10:42:05Z",
    "validation_pass": 4,
    "operations_hash": "LLoaFXgbRsSLWTuVoZpDzW7PreL2AdgSbNGDXz9Z7XHkymxHqXNq8",
    "fitness": [
      "00",
      "0000000000000004"
    ],
    "context": "CoWHBGGQzf1zCA3mJ668UP5JgcCNUtRYz1yqqSK124kkB5CYAfs1",
    "priority": 12,
    "proof_of_work_nonce": "0000000368a75719",
    "signature": "sigVLNCCBzBhGYKqr7wFhMhyHLoFaZBpU1qKYpNmXYGVmMs5XR7Qr2mnpotX5je4yyjR7KakjLPRxaJ4qMSyHNdnSk4oDoVn"
  },
  "metadata": {
    "protocol": "PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP",
    "next_protocol": "PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP",
    "test_chain_status": {
      "status": "not_running"
    },
    "max_operatio

              }
            }
          }
        ],
        "signature": "sigPv1pGYf31XmcQZxPQLzRPyDYsHUKMTeVMuMNAuCAyVkFmhiaY1AJkBUPG7DuDBup9Jmbs4Xa9uDU4bukZwRLfmyQyNTw8"
      }
    ]
  ]
}


## Implementation of application front-end
For the implementation of front-end we will use Eztz library (https://github.com/TezTech/eztz). This library is implementing RPC communication with Tezos and also provides functionality for key generation, signing, verification, and contract interaction.

For the sample we will fetch library from github (At the time of writing Eztz does not provide NPM package).



In [ ]:
cd ~/tezos-dev/calculator-dapp

#this will create default npm module
npm init -y

#this will download eztz library for CLI
wget https://github.com/TezTech/eztz/raw/master/dist/eztz.cli.js

#this will download eztz dependencies required for CLI
npm install --save xmlhttprequest
npm install --save bs58check
npm install --save libsodium-wrappers
npm install --save bip39

Wrote to /Users/martinkovacik/tezos-dev/calculator-dapp/package.json:

{
  "name": "calculator-dapp",
  "version": "1.0.0",
  "description": "",
  "main": "eztz.cli.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}




   ╭───────────────────────────────────────────────────────────────╮
   │                                                               │
   │       New minor version of npm available! 6.4.1 → 6.5.0       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v6.5.0   │
   │               Run npm install -g npm to update!               │
   │                                                               │
   ╰───────────────────────────────────────────────────────────────╯

--2019-01-16 11:42:16--  https://github.com/TezTech/eztz/raw/master/dist/eztz.cli.js
Resolving github.com (github.com)... 140.82.118.3, 140.82.118.4
Prip'ajanie k github.com (github.com)|140.82.118.3|:443... c

To be able to interact with our smart contract, we need to have account - this is represented by private/public key pair. For the demo we will create account in JS application - this is the activity we did earlier as [Testing Account Creation](#Testing-account-creation).

    TBD - continue with content

## Sandbox Clean-up

Command to terminate sandbox blockchain running in the background

In [ ]:
kill $(<$TEZOS_HOME/tezos_sandbox_pid.txt)